# LAB 5 MOS: Optimización Multiobjetivo

- Samuel Augusto Hernandez 202213772
- Tomas Alberto Rodriguez 2022128686

## Problema 1: Optimización Multiobjetivo en Distribución de Recursos para Misión Humanitaria

### Formulación del modelo Multiobjetivo

#### Conjuntos

$$ R = \{R_1,R_2,R_3,R_4,R_5\}$$
$$ A = \{A_1,A_2,A_3,A_4\}$$
$$ Z = \{Z_1,Z_2,Z_3,Z_4\}$$
$$ V = \{V_1,V_2\} $$

#### Parámetros

Recursos:

Valor de impacto (miles USD/TON):
$$ Va_r: \forall r \in R $$

Peso(TON/unidad):
$$ W_r: \forall r \in R$$

Volumen(m^3/unidad):
$$ V_r: \forall r \in R$$

Disponibilidad(unidades):
$$ D_r: \forall r \in R$$

Aviones:

Capacidad Peso (TON):
$$ CP_a: \forall a \in A$$

Capacidad Volumen (m^3):
$$ CV_a: \forall a \in A$$

Costo Fijo (miles UDS):
$$ CF_a: \forall a \in A$$

Costo Variable (miles UDS/km):
$$ C_a: \forall a \in A$$

Zonas:

Distancias (km):
$$ D_z: \forall z \in Z$$

Poblacion (miles):
$$ P_z: \forall z \in Z$$

Multiplicador:
$$ M_z: \forall z \in Z$$

Demanda de recurso por zona (TON):
$$ D_{r,z,v}: \forall r \in R, \forall z \in Z $$

#### Variables de decision

Asignación de aviones a zonas:

$$ x_{a,z,v} \in N, \forall a \in A, \forall z \in Z, \forall v \in V $$

Asignación de recursos a aviones:

$$ y_{a,r,v} \in N, \forall a \in A, \forall r \in R, \forall V \in V $$

#### Funciones objetivo

Maximizar Impacto social:
$$ Z_1 = \sum_{r\in R} \sum_{a\in A} \sum_{v\in V} \sum_{z\in Z} Va_r \cdot y_{a,r,v} \cdot M_z $$

Minimizar Costo de tranporte:
$$ Z_2 = \sum_{a\in A} \sum_{z\in Z} \sum_{v\in V} CF_a \cdot x_{a,z,v} + \sum_{v\in V} \sum_{z\in Z} \sum_{a\in A} C_a \cdot x_{a,z,v} \cdot D_z $$


#### Restricciones

Capacidad de peso y volumen de los aviones:
$$ \sum_{r\in R} \sum_{v\in V} y_{a,r,v} \cdot W_r \leq CP_a, \forall a \in A $$
$$ \sum_{r\in R} \sum_{v\in V} y_{a,r,v} \cdot V_r \leq CV_a, \forall a \in A $$

Asignacion de recursos no sobrepase la disponibilidad:
$$ \sum_{a\in A} \sum_{v\in V} y_{a,r,v} \leq D_r, \forall r \in R $$

Seguridad de los medicamentos:
$$ y_{1,medicamento,v} = 0, \forall v \in V $$

Compatibilidad de recursos:
$$ y_{a,medicamentos,v} + y_{a,agua,v} $$

Asignacion de zonas por avion y viaje (el avion solo puede ir una vez a una zona):
$$ \sum_{z\in Z} \sum_{v\in V} x_{a,z,v} \leq 1, \forall a \in A $$

Satisfaccion de necesidades:
$$ \sum_{a\in A} \sum_{v\in V} y_{a,r,v} \geq D_z, \forall z \in Z, \forall r \in R $$

Solo lleva recursos si va a la zona:
$$ x_{a,z,v} \cdot BigM \geq \sum_{r \in R} y_{a,r,v}, \forall z \in Z, \forall v \in V, \forall a \in A $$

No negatividad y binaria:
$$ x_{a,z,v} \in {0,1} $$
$$ y_{a,r,v} \geq 0  \in N$$

#### Modelo de optimización


### Implementación del modelo

In [1]:
recursos = ['alimentos', 'medicinas', 'equipo_medico', 'agua', 'mantas'] 
aviones = ['A1', 'A2', 'A3', 'A4']  
zonas = ['A','B','C','D']

datos_recursos = {
    'alimentos': {'valor': 50, 'peso': 5, 'volumen': 3, 'disponibilidad': 12},
    'medicinas': {'valor': 100, 'peso': 2, 'volumen': 1, 'disponibilidad': 15},
    'equipo_medico': {'valor': 120, 'peso': 0.3, 'volumen': 0.5, 'disponibilidad': 40},
    'agua': {'valor': 60, 'peso': 6, 'volumen': 4, 'disponibilidad': 15},
    'mantas': {'valor': 40, 'peso': 3, 'volumen': 2, 'disponibilidad': 20},
}

datos_aviones = {
    'A1': {'peso_max': 40, 'volumen_max': 35, 'costo_fijo': 15, 'costo_variable': 0.020},
    'A2': {'peso_max': 50, 'volumen_max': 40, 'costo_fijo': 20, 'costo_variable': 0.025},
    'A3': {'peso_max': 60, 'volumen_max': 45, 'costo_fijo': 25, 'costo_variable': 0.030},
    'A4': {'peso_max': 45, 'volumen_max': 38, 'costo_fijo': 18, 'costo_variable': 0.022}
}

datos_zonas = {
    'A': {'distancia': 40, 'poblacion': 35, 'multiplicador': 15},
    'B': {'distancia': 50, 'poblacion': 40, 'multiplicador': 20},
    'C': {'distancia': 60, 'poblacion': 45, 'multiplicador': 25},
    'D': {'distancia': 45, 'poblacion': 38, 'multiplicador': 18}
}

In [5]:
from pyomo.environ import *
import numpy as np

# Datos del problema
recursos = ['alimentos', 'medicinas', 'equipo_medico', 'agua', 'mantas']
aviones = ['A1', 'A2', 'A3', 'A4']
zonas = ['A', 'B', 'C', 'D']
viajes = ['V1', 'V2']

# Diccionarios de datos
datos_recursos = {
    'alimentos': {'valor': 50, 'peso': 5, 'volumen': 3, 'disponibilidad': 12},
    'medicinas': {'valor': 100, 'peso': 2, 'volumen': 1, 'disponibilidad': 15},
    'equipo_medico': {'valor': 120, 'peso': 0.3, 'volumen': 0.5, 'disponibilidad': 40},
    'agua': {'valor': 60, 'peso': 6, 'volumen': 4, 'disponibilidad': 15},
    'mantas': {'valor': 40, 'peso': 3, 'volumen': 2, 'disponibilidad': 20},
}

datos_aviones = {
    'A1': {'peso_max': 40, 'volumen_max': 35, 'costo_fijo': 15, 'costo_variable': 0.020},
    'A2': {'peso_max': 50, 'volumen_max': 40, 'costo_fijo': 20, 'costo_variable': 0.025},
    'A3': {'peso_max': 60, 'volumen_max': 45, 'costo_fijo': 25, 'costo_variable': 0.030},
    'A4': {'peso_max': 45, 'volumen_max': 38, 'costo_fijo': 18, 'costo_variable': 0.022}
}

datos_zonas = {
    'A': {'distancia': 40, 'poblacion': 35, 'multiplicador': 15},
    'B': {'distancia': 50, 'poblacion': 40, 'multiplicador': 20},
    'C': {'distancia': 60, 'poblacion': 45, 'multiplicador': 25},
    'D': {'distancia': 45, 'poblacion': 38, 'multiplicador': 18}
}

# Crear un modelo
model = ConcreteModel()

# Conjuntos
model.R = Set(initialize=recursos)
model.A = Set(initialize=aviones)
model.Z = Set(initialize=zonas)
model.V = Set(initialize=viajes)

# Parámetros
model.Va = Param(model.R, initialize={r: datos_recursos[r]['valor'] for r in recursos})
model.W = Param(model.R, initialize={r: datos_recursos[r]['peso'] for r in recursos})
model.V_r = Param(model.R, initialize={r: datos_recursos[r]['volumen'] for r in recursos})
model.D_r = Param(model.R, initialize={r: datos_recursos[r]['disponibilidad'] for r in recursos})

model.CP = Param(model.A, initialize={a: datos_aviones[a]['peso_max'] for a in aviones})
model.CV = Param(model.A, initialize={a: datos_aviones[a]['volumen_max'] for a in aviones})
model.CF = Param(model.A, initialize={a: datos_aviones[a]['costo_fijo'] for a in aviones})
model.C = Param(model.A, initialize={a: datos_aviones[a]['costo_variable'] for a in aviones})

model.D_z = Param(model.Z, initialize={z: datos_zonas[z]['distancia'] for z in zonas})
model.M_z = Param(model.Z, initialize={z: datos_zonas[z]['multiplicador'] for z in zonas})

# Variables de decisión
model.x = Var(model.A, model.Z, model.V, within=Binary)  # Asignación de aviones a zonas por viaje
model.y = Var(model.A, model.R, model.V, within=NonNegativeReals)  # Cantidad de recursos transportados por avión

# Función Objetivo 1: Maximizar impacto social
model.f1 = sum(model.Va[r] * model.y[a, r, v] * model.M_z[z] for r in model.R for a in model.A for v in model.V for z in model.Z)

# Función Objetivo 2: Minimizar costo de transporte
model.f2 = sum(model.CF[a] * model.x[a, z, v] for a in model.A for z in model.Z for v in model.V) + \
    sum(model.C[a] * model.x[a, z, v] * model.D_z[z] * model.y[a, r, v] for a in model.A for r in model.R for z in model.Z for v in model.V)

# Normalización: Calcular los valores máximos de f1 y f2 para normalizar
max_f1 = sum(model.Va[r] * model.D_r[r] * model.M_z[z] for r in model.R for z in model.Z)
max_f2 = sum(model.CF[a] * model.D_z[z] * model.D_r[r] for a in model.A for z in model.Z for r in model.R)

# Proceso de Suma Ponderada

w2_vec = np.linspace(0, 1, num=11)  # Generar valores de ponderación entre 0 y 1

f1_vec = []  # Para almacenar el valor de Z1 (impacto social)
f2_vec = []  # Para almacenar el valor de Z2 (costo de transporte)

for w2 in w2_vec:
    w1 = 1 - w2
    
    # Función objetivo general con ponderación
    model.O_z = Objective(expr=w1*(max_f1 - model.f1) + w2*(model.f2), sense=minimize)
    
    # Restricciones:

    # Restricción de capacidad de peso y volumen de los aviones
    def restriccion_capacidad_peso(model, a):
        return sum(model.y[a, r, v] * model.W[r] for r in model.R for v in model.V) <= model.CP[a]
    model.restriccion_capacidad_peso = Constraint(model.A, rule=restriccion_capacidad_peso)

    def restriccion_capacidad_volumen(model, a):
        return sum(model.y[a, r, v] * model.V_r[r] for r in model.R for v in model.V) <= model.CV[a]
    model.restriccion_capacidad_volumen = Constraint(model.A, rule=restriccion_capacidad_volumen)

    # Restricción de disponibilidad de los recursos
    def restriccion_disponibilidad(model, r):
        return sum(model.y[a, r, v] for a in model.A for v in model.V) <= model.D_r[r]
    model.restriccion_disponibilidad = Constraint(model.R, rule=restriccion_disponibilidad)

    # Restricción de seguridad de los medicamentos (medicinas no se pueden transportar en el avión A1)
    def restriccion_medicinas(model, v):
        return model.y['A1', 'medicinas', v] == 0
    model.restriccion_medicinas = Constraint(model.V, rule=restriccion_medicinas)

    # Restricción de compatibilidad de recursos (agua y medicamentos no pueden viajar en el mismo avión)
    def restriccion_compatibilidad(model, a, v):
        return model.y[a, 'medicinas', v] + model.y[a, 'agua', v] <= 1
    model.restriccion_compatibilidad = Constraint(model.A, model.V, rule=restriccion_compatibilidad)

    # Restricción de asignación de zonas por avión y viaje
    def restriccion_zona_unica(model, a):
        return sum(model.x[a, z, v] for z in model.Z for v in model.V) <= 1
    model.restriccion_zona_unica = Constraint(model.A, rule=restriccion_zona_unica)

    # Restricción de satisfacción de las necesidades mínimas
    def restriccion_necesidades(model, z, r):
        return sum(model.y[a, r, v] for a in model.A for v in model.V) >= model.D_r[r]
    model.restriccion_necesidades = Constraint(model.Z, model.R, rule=restriccion_necesidades)

    # Restricción de transporte solo si el avión va a la zona
    def restriccion_transporte(model, a, z, v):
        return model.x[a, z, v] * 100 >= sum(model.y[a, r, v] for r in model.R)
    model.restriccion_transporte = Constraint(model.A, model.Z, model.V, rule=restriccion_transporte)
    
    # Resolver el modelo
    SolverFactory('glpk').solve(model)
    
    # Obtener los valores de las funciones objetivo
    valor_f1 = value(model.f1)
    valor_f2 = value(model.f2)
    
    # Almacenar los resultados
    f1_vec.append(valor_f1)
    f2_vec.append(valor_f2)
    
    # Eliminar los componentes del modelo para la próxima iteración
    delete_component(model, 'O_z')
    delete_component(model, 'restriccion_capacidad_peso')
    delete_component(model, 'restriccion_capacidad_volumen')
    delete_component(model, 'restriccion_disponibilidad')
    delete_component(model, 'restriccion_medicinas')
    delete_component(model, 'restriccion_compatibilidad')
    delete_component(model, 'restriccion_zona_unica')
    delete_component(model, 'restriccion_necesidades')
    delete_component(model, 'restriccion_transporte')

# Mostrar resultados (frente de Pareto)
for i in range(len(f1_vec)):
    print(f"Peso w2 = {w2_vec[i]:.2f}, Impacto Social = {f1_vec[i]:.2f}, Costo de Transporte = {f2_vec[i]:.2f}")


ERROR: evaluating object as numeric value: y[A1,alimentos,V1]
        (object: <class 'pyomo.core.base.var.VarData'>)
    No value for uninitialized NumericValue object y[A1,alimentos,V1]
ERROR: evaluating object as numeric value: 750*y[A1,alimentos,V1] +
1000*y[A1,alimentos,V1] + 1250*y[A1,alimentos,V1] + 900*y[A1,alimentos,V1] +
750*y[A1,alimentos,V2] + 1000*y[A1,alimentos,V2] + 1250*y[A1,alimentos,V2] +
900*y[A1,alimentos,V2] + 750*y[A2,alimentos,V1] + 1000*y[A2,alimentos,V1] +
1250*y[A2,alimentos,V1] + 900*y[A2,alimentos,V1] + 750*y[A2,alimentos,V2] +
1000*y[A2,alimentos,V2] + 1250*y[A2,alimentos,V2] + 900*y[A2,alimentos,V2] +
750*y[A3,alimentos,V1] + 1000*y[A3,alimentos,V1] + 1250*y[A3,alimentos,V1] +
900*y[A3,alimentos,V1] + 750*y[A3,alimentos,V2] + 1000*y[A3,alimentos,V2] +
1250*y[A3,alimentos,V2] + 900*y[A3,alimentos,V2] + 750*y[A4,alimentos,V1] +
1000*y[A4,alimentos,V1] + 1250*y[A4,alimentos,V1] + 900*y[A4,alimentos,V1] +
750*y[A4,alimentos,V2] + 1000*y[A4,alimentos,V2] + 1250

ValueError: No value for uninitialized NumericValue object y[A1,alimentos,V1]

#### Visualizacion del frente de Pareto

In [ ]:
#codigo

### Analisis

## Problema 2: Optimización Multiobjetivo en Planificación de Rutas de Inspección

### Formulación del modelo Multiobjetivo

#### Conjuntos

#### Parámetros

#### Variables de decision

#### Funciones objetivo

#### Restricciones

### Modelo de optimización


### Implementación del modelo

In [ ]:
#codigo

#### Visulizacion del frente de Pareto

In [ ]:
#codigo

### Analisis